In [14]:
import pandas as pd

def procesar_calendario_ciclos(ruta_archivo: str, fila_inicio: int = 2, columna_inicio: str = 'N', columna_final: str = 'BP') -> pd.DataFrame:
    """
    Procesa un archivo Excel con formato de calendario de ciclos.
    
    Parámetros:
        ruta_archivo (str): Ruta al archivo .xlsx
        fila_inicio (int): Fila donde comienzan los nombres de columna (índice 0-based). Ej: fila 3 → valor 2
        columna_inicio (str): Letra de la columna donde comienza la tabla. Ej: 'N'
        
    Retorna:
        pd.DataFrame: DataFrame en formato largo con Gerencia, Sector, Ciclo, Fechas y Días
    """
    # Determinar rango de columnas (N:BP contiene los ciclos hasta el 19)
    columna_fin = columna_final
    df_raw = pd.read_excel(ruta_archivo, header=fila_inicio, usecols=f'{columna_inicio}:{columna_fin}')

    # Detectar columnas fijas reales
    columnas_posibles = df_raw.columns.tolist()
    columnas_fijas = [col for col in columnas_posibles if col.startswith('Gerencia') or col.startswith('Cod Sector') 
                      or col.startswith('Sector') or col.startswith('Bloque')][:4]

    # Ciclos disponibles
    ciclos = [f'{i:02}' for i in range(3, 20)]  # 03 a 19

    datos = []
    for _, fila in df_raw.iterrows():
        base = {
            'Gerencia': fila.get(columnas_fijas[0]),
            'Cod Sector': fila.get(columnas_fijas[1]),
            'Sector': fila.get(columnas_fijas[2]),
            'Bloque': fila.get(columnas_fijas[3]),
        }
        for i, ciclo in enumerate(ciclos, start=2):  # "Días.2" para ciclo 03
            nombre_inicio = f'Fecha Inicio Ciclo {ciclo}'
            nombre_termino = f'Fecha Termino Ciclo {ciclo}'
            nombre_dias = f'Días.{i}'
            if nombre_inicio in fila and nombre_termino in fila and nombre_dias in fila:
                datos.append({
                    **base,
                    'Ciclo': ciclo,
                    'Fecha Inicio': pd.to_datetime(fila[nombre_inicio], errors='coerce'),
                    'Fecha Termino': pd.to_datetime(fila[nombre_termino], errors='coerce'),
                    'Días': fila[nombre_dias]
                })

    # Convertir a DataFrame final
    df_final = pd.DataFrame(datos)
    df_final.sort_values(by=['Gerencia', 'Cod Sector', 'Sector', 'Bloque', 'Ciclo'], inplace=True)
    df_final.reset_index(drop=True, inplace=True)

    return df_final


In [15]:
df = procesar_calendario_ciclos(r"C:\Users\Spider Build\Downloads\Calendario 2025 r03 (1).xlsx")
df

,Gerencia,Cod Sector,Sector,Bloque,Ciclo,Fecha Inicio,Fecha Termino,Días
0,ARAUCARIA,200000500.0,Peumo,3.0,03,2025-01-23,2025-02-12,21.0
1,ARAUCARIA,200000500.0,Peumo,3.0,04,2025-02-13,2025-03-05,21.0
2,ARAUCARIA,200000500.0,Peumo,3.0,05,2025-03-06,2025-03-26,21.0
3,ARAUCARIA,200000500.0,Peumo,3.0,06,2025-03-27,2025-04-16,21.0
4,ARAUCARIA,200000500.0,Peumo,3.0,07,2025-04-17,2025-04-30,14.0
...,...,...,...,...,...,...,...,...
1593,NaT,NaT,NaT,NaT,19,NaT,NaT,NaT
1594,NaT,NaT,NaT,NaT,19,NaT,NaT,NaT
1595,NaT,NaT,NaT,NaT,19,NaT,NaT,NaT
1596,NaT,NaT,NaT,NaT,19,NaT,NaT,NaT


In [13]:
df.to_excel(r"C:\Users\Spider Build\Downloads\Calendario2025_outputv2.xlsx")